In [1]:
import pandas as pd
from IPython.display import display
import numpy as np
from collections import Counter
import nltk
import psycopg2
from nltk.corpus import stopwords
import re
from itertools import chain 
from IPython.display import Image
import math
import time
import logging 
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
File = 'F:\Pretrained Models\glove.twitter.27B\glove.twitter.27B.200d.txt'
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r',encoding="utf8")
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel
def similarity(model,word1,word2):
    vec1 = [model[word1]]
    vec2 = [model[word2]]
    similarity_score = cosine_similarity(X =vec1, Y=vec2, dense_output=True)
    return similarity_score[0][0]
gloveModel = loadGloveModel(File)

Loading Glove Model
1193514  words loaded!


In [4]:
def process(array):
    text = re.sub(r'\[[0-9]*\]',' ',str(array))  #Remove Numbers
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text) # Remove nums
    text = re.sub(r'\s+',' ',text)  #Remove extra space
    text = re.sub(r"[^a-zA-Z0-9]+",' ',text)  #Remove special characters
    text = text.lower()  #Lower case all
    text = nltk.sent_tokenize(text)  #Tokenize to sentences 
    keywords = [nltk.word_tokenize(sentence) for sentence in text]
    for i in range(len(keywords)):
        keywords[i] = [word for word in keywords[i] if word not in stopwords.words('english')]
    return keywords

#Frame pre-processing function
def prepro(frame):
    frame2 = frame
    for c in range(len(frame2)):
        keywords = process(frame2['Caption'][c])
        frame2['Caption'][c] = keywords
    return frame2

def flat(array):     # 2D array reduction
    flatten_list = [j for sub in array for j in sub] 
    flatten_list = list(chain.from_iterable(flatten_list))
    return flatten_list


# normalize() -> given an array, converts to 1/0, top int(pos) will be 1
def normalize(keys, pos =3):  
    ax = [i for i in keys]
    temp = [i for i in keys]
    temp.sort()
    temp = temp[-pos:]
    for x in temp:
        ax[keys.index(x)] = 1
    for x in range(len(ax)):
        if ax[x] != 1:
            ax[x] = 0
    return ax

# compute() => category[] to be called outside
def compute(caption,category,top =3):
    ar = []
    score = []
    
    # Code to get frequency distribution and unique keywords array
    counts = Counter(caption)
    labels, values = zip(*counts.items())
    ## sort your values in descending order
    indSort = np.argsort(values)[::-1]
    ## rearrange your data
    keywords = np.array(labels)[indSort]  # Label
    caption_freq = np.array(values)[indSort]  # Values
    
    # Detect words not in Google Dict | Put freq = 0
    for x in keywords:
        try:
            restConst = similarity(gloveModel,x,'something')
        except KeyError:
            caption_freq[np.where(keywords == x)] = 0
        
    #Google similaity function
    for x in category:
        empty = []
        for y in keywords:
            try:
                empty.append(similarity(gloveModel,x,y))
            except:
                empty.append(0)
        ar.append(empty)
    
    # Store the similarity values in dataframe
    frame = pd.DataFrame(ar, columns = keywords)
    
    #Normalize | top select
    for key in frame.columns:
        frame[key] = normalize(frame[key].tolist(),top)
    
    # Multiply with frequency
    for row in range(len(frame)):
        frame.values[row] = [i*j for i,j in zip(frame.values[row],caption_freq)]
    # Sum the values => Score
    for row in range(len(frame)):
        score.append(sum(frame.values[row]))
    
    frame['category'] = category
    frame['Scores'] = score
    return frame

def deviation(array):
    mu = max(array)
    l = len(array)
    ar = []
    for x in range(l):
        ar.append(math.sqrt((array[x]-mu)**2)/l)
    total = sum(ar)
    for x in range(l):
        ar[x] = (ar[x]/total)*100
    return ar

def sort_cat(dev,cat, thre = 2):
    final_cat = []
    for i in range(len(dev)):
        if dev[i] <=thre:
            final_cat.append(cat[i])
    return final_cat

def get_row_pscore(col_name,f1,f2):  # f1-mainframe | f2-frame
    ud = f1.at[0,'user_id']
    hd = f1.at[0,'handle']
    fl = f1.at[0,'followers']
    ul = f1.at[0,'url']
    row_in_array = [ud,hd,fl,ul]
    dev_array = f2['Deviation'].tolist()
    row_in_array.extend(dev_array)
    zip_it = zip(col_name,row_in_array)
    convert_to_dict = dict(zip_it)
    return convert_to_dict
def top_category(f2,categories,thre=2):
    final_cat = [0]*len(categories)
    dev_scores = f2['Deviation'].tolist()
    rank_of_cat = 1
    for i in range(len(dev)):
        if dev_scores[i] <=thre:
            final_cat[i] = rank_of_cat 
            rank_of_cat = rank_of_cat +1
    return final_cat
def get_row_result(col_name,f1,rank_array):  # f1-mainframe
    ud = f1.at[0,'user_id']
    hd = f1.at[0,'handle']
    fl = f1.at[0,'followers']
    ul = f1.at[0,'url']
    row_in_array = [ud,hd,fl,ul]
    row_in_array.extend(rank_array)
    zip_it = zip(col_name,row_in_array)
    convert_to_dict = dict(zip_it)
    return convert_to_dict

In [5]:
conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
cur = conn.cursor()

### Get IDs | <500,000 & India   |   Top 10k only

In [8]:
# cur.execute('''SELECT user_id, handle, url, followers FROM instagram_accounts WHERE (followers < 500000 and country = 'India') ORDER BY followers DESC''')
cur.execute('''SELECT  "instagram_accounts"."user_id", "instagram_accounts"."handle", "instagram_accounts"."url", "instagram_accounts"."followers" FROM "instagram_accounts" WHERE (followers < 500000 and country = 'India') ORDER BY "instagram_accounts"."followers" DESC''')
InstagramAccount = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])
InstagramAccount = InstagramAccount[5000:6000]
Insta_ids = InstagramAccount['user_id'].tolist()

In [10]:
# Categories
categories = ['food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal']


# DataFrame Creation

profilescore = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
onefive = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
twofive = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])
threefive = pd.DataFrame(columns = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal'])

# This required to run some function

col_name = ['user_id','handle','followers','url','food', 'fashion', 'makeup', 'beauty', 'lifestyle','luxury', 'traveler', 'art','photography','fitness','sports','gaming', 'entertainment', 'technology', 'finance','education', 'animal']


In [11]:
#Timer start
tic = time.perf_counter()
prev_tac = tic
for x in Insta_ids:
    try:
        tac = time.perf_counter()
        time_compare = tac-prev_tac
        if time_compare >= 1000:
            conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
            cur = conn.cursor()
            prev_tac = tac
        userid =x
        # Id, handle, url
        cur.execute('SELECT user_id, handle, url, followers  FROM instagram_accounts WHERE user_id = '+str(userid)+' ORDER BY user_id')
        mainframe = pd.DataFrame()
        mainframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])
        # captions
        cur.execute('SELECT "insta_user_posts"."insta_user_id", "insta_user_posts"."caption" FROM "insta_user_posts" WHERE (insta_user_id = '+str(userid)+')')
        randomframe = pd.DataFrame()
        randomframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "Caption"])
        if len(randomframe) ==0:
            raise Exception("No captions in Winkl database")
        r2 = pd.DataFrame()
        r2 = prepro(randomframe)
        # Merge
        r3 = pd.DataFrame()
        r3 = r2.groupby('user_id').agg(lambda x: x.tolist())
        caption_array = r3['Caption'][x]
        # to 1D
        caption_array = flat(caption_array)
        # Word2vec computation
        frame = pd.DataFrame()
        frame = pd.DataFrame(compute(caption_array,categories,3))
        #Convert to Percentage
        per = frame['Scores'].tolist()
        per_sum = sum(per)
        for x in range(len(per)):
            per[x] = round((per[x]/per_sum)*100)
        frame['Percentage'] = per
        # Deviation of categories
        dev = deviation(frame['Scores'].tolist())
        frame['Deviation'] = dev
        # store profile score
        row_df_1 = get_row_pscore(col_name,mainframe,frame)
        profilescore = profilescore.append(row_df_1,ignore_index=True)

        # store onefive result
        select_15_array = top_category(frame,categories,thre=1.5)
        row_df_2 = get_row_result(col_name,mainframe,select_15_array)
        onefive = onefive.append(row_df_2,ignore_index=True)

        # store twofive result
        select_25_array = top_category(frame,categories,thre=2.5)
        row_df_3 = get_row_result(col_name,mainframe,select_25_array)
        twofive = twofive.append(row_df_3,ignore_index=True)

        # store threefive result
        select_35_array = top_category(frame,categories,thre=3.5)
        row_df_4 = get_row_result(col_name,mainframe,select_35_array)
        threefive = threefive.append(row_df_4,ignore_index=True)
    except Exception as Argument:
        
        # creating/opening a file 
        f = open("E:\Winkl Mains\DATA\glove model testing\glove_10k_profile\errorlogs2.txt", "a") 
        # writing in the file 
        f.write("In user_id:"+str(userid)+": "+str(Argument)+str("\n")) 
        # closing the file 
        f.close()  

display(profilescore)
display(onefive)
display(twofive)
display(threefive)

# Timer end
toc = time.perf_counter()
print(f"Ran the model in {toc - tic:0.4f} seconds")

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is

C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\acer\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is

...

,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,211277,thebrunettewave,51276,https://www.instagram.com/thebrunettewave?utm_...,2.381770,0.000000,5.937250,2.061147,4.059315,6.595672,...,6.326577,6.549868,5.983053,3.423795,7.929692,8.473606,7.557540,8.971716,7.185389,8.370548
1,61462,skyplanners,51266,https://www.instagram.com/skyplanners/,3.253796,4.154013,8.362256,4.056399,5.054230,0.000000,...,5.921909,3.622560,8.318872,6.800434,7.462039,6.420824,7.809111,5.889371,7.678959,8.373102
2,299,_mihirjariwala_,51262,https://www.instagram.com/_mihirjariwala_/,0.000000,4.097850,5.697517,2.280046,6.419445,6.403864,...,5.707905,6.029916,8.055469,3.204529,8.206087,7.655552,5.863717,8.195700,5.832554,8.164537
3,233120,reemasharma1313,51261,https://www.instagram.com//reemasharma1313/,6.384743,0.000000,4.264392,1.717602,4.655295,7.782516,...,6.266288,1.054253,7.202085,6.704572,7.936508,7.675906,7.628524,8.066809,7.664061,7.818053
4,134285,kings_works,51251,https://www.instagram.com/kings_works/,0.000000,3.795222,8.295828,5.139634,7.486585,9.006982,...,1.557902,6.019560,8.820899,0.379378,7.619295,4.484738,5.556517,7.890485,6.173908,8.344873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,179218,ashishrsharma,41487,https://www.instagram.com/ashishrsharma/,0.000000,5.753958,7.342317,1.875288,6.819696,7.414049,...,5.267203,7.070759,8.018650,3.161347,7.557514,6.184352,5.882052,7.721474,4.524261,7.562638
978,86422,hellomommyinme,41487,https://www.instagram.com/hellomommyinme/,0.000000,6.842956,6.453204,2.868617,6.613467,7.019815,...,5.639559,6.865241,7.464569,3.177764,7.364997,7.315685,5.306231,7.578840,4.851045,7.105162
979,4981,shadesofmoon,41484,https://www.instagram.com/shadesofmoon/,0.000000,4.480334,4.922340,2.420215,6.280813,7.025732,...,5.623986,5.987173,7.551194,3.604049,7.625377,7.554285,6.960822,7.994745,6.605363,7.433738
980,3178,never_stay_sober,41477,https://www.instagram.com/never_stay_sober/,0.000000,4.840774,5.316076,2.274140,6.507989,6.906512,...,5.469635,7.239223,7.703557,2.950532,7.462250,7.326972,6.902855,7.955833,6.390991,6.990604


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,211277,thebrunettewave,51276,https://www.instagram.com/thebrunettewave?utm_...,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,61462,skyplanners,51266,https://www.instagram.com/skyplanners/,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,299,_mihirjariwala_,51262,https://www.instagram.com/_mihirjariwala_/,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,233120,reemasharma1313,51261,https://www.instagram.com//reemasharma1313/,0,1,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
4,134285,kings_works,51251,https://www.instagram.com/kings_works/,1,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,179218,ashishrsharma,41487,https://www.instagram.com/ashishrsharma/,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
978,86422,hellomommyinme,41487,https://www.instagram.com/hellomommyinme/,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
979,4981,shadesofmoon,41484,https://www.instagram.com/shadesofmoon/,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
980,3178,never_stay_sober,41477,https://www.instagram.com/never_stay_sober/,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,211277,thebrunettewave,51276,https://www.instagram.com/thebrunettewave?utm_...,1,2,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,61462,skyplanners,51266,https://www.instagram.com/skyplanners/,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,299,_mihirjariwala_,51262,https://www.instagram.com/_mihirjariwala_/,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
3,233120,reemasharma1313,51261,https://www.instagram.com//reemasharma1313/,0,1,0,2,0,0,...,0,3,0,0,0,0,0,0,0,0
4,134285,kings_works,51251,https://www.instagram.com/kings_works/,1,0,0,0,0,0,...,2,0,0,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,179218,ashishrsharma,41487,https://www.instagram.com/ashishrsharma/,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
978,86422,hellomommyinme,41487,https://www.instagram.com/hellomommyinme/,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
979,4981,shadesofmoon,41484,https://www.instagram.com/shadesofmoon/,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
980,3178,never_stay_sober,41477,https://www.instagram.com/never_stay_sober/,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0


,user_id,handle,followers,url,food,fashion,makeup,beauty,lifestyle,luxury,...,art,photography,fitness,sports,gaming,entertainment,technology,finance,education,animal
0,211277,thebrunettewave,51276,https://www.instagram.com/thebrunettewave?utm_...,1,2,0,3,0,0,...,0,0,0,4,0,0,0,0,0,0
1,61462,skyplanners,51266,https://www.instagram.com/skyplanners/,1,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2,299,_mihirjariwala_,51262,https://www.instagram.com/_mihirjariwala_/,1,0,0,2,0,0,...,0,0,0,3,0,0,0,0,0,0
3,233120,reemasharma1313,51261,https://www.instagram.com//reemasharma1313/,0,1,0,2,0,0,...,0,3,0,0,0,0,0,0,0,0
4,134285,kings_works,51251,https://www.instagram.com/kings_works/,1,0,0,0,0,0,...,2,0,0,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977,179218,ashishrsharma,41487,https://www.instagram.com/ashishrsharma/,1,0,0,2,0,0,...,0,0,0,3,0,0,0,0,0,0
978,86422,hellomommyinme,41487,https://www.instagram.com/hellomommyinme/,1,0,0,2,0,0,...,0,0,0,3,0,0,0,0,0,0
979,4981,shadesofmoon,41484,https://www.instagram.com/shadesofmoon/,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
980,3178,never_stay_sober,41477,https://www.instagram.com/never_stay_sober/,1,0,0,2,0,0,...,0,0,0,3,0,0,0,0,0,0


Ran the model in 8091.6896 seconds


In [12]:
profilescore.to_csv(r'E:\Winkl Mains\DATA\glove model testing\glove_10k_profile\profile_deviation_score.csv')
onefive.to_csv(r'E:\Winkl Mains\DATA\glove model testing\glove_10k_profile\categorised_1_5.csv')
twofive.to_csv(r'E:\Winkl Mains\DATA\glove model testing\glove_10k_profile\categorised_2_5.csv')
threefive.to_csv(r'E:\Winkl Mains\DATA\glove model testing\glove_10k_profile\categorised_3_5.csv')
# Ran the model in 8091.6896 seconds

### END 

## TESTING SECTION

userid = 300226
cur.execute('SELECT user_id, handle, url, followers  FROM instagram_accounts WHERE user_id = '+str(userid)+' ORDER BY user_id')
mainframe = pd.DataFrame()
mainframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "handle", "url", "followers"])


cur.execute('SELECT "insta_user_posts"."insta_user_id", "insta_user_posts"."caption" FROM "insta_user_posts" WHERE (insta_user_id = '+str(userid)+')')
randomframe = pd.DataFrame()
randomframe = pd.DataFrame(cur.fetchall(),columns=["user_id", "Caption"])
r2 = pd.DataFrame()
r2 = prepro(randomframe)
# Merge
r3 = pd.DataFrame()
r3 = r2.groupby('user_id').agg(lambda x: x.tolist())
cont = r3['Caption'][userid]
cont = flat(cont)
frame = pd.DataFrame()
frame = pd.DataFrame(compute(cont,categories,3))
#Convert to Percentage
per = frame['Scores'].tolist()
per_sum = sum(per)
for x in range(len(per)):
    per[x] = round((per[x]/per_sum)*100)
frame['Percentage'] = per
# Deviation of categories
dev = deviation(frame['Scores'].tolist())
frame['Deviation'] = dev

# store profile score
row_df_1 = get_row_pscore(col_name,mainframe,frame)
profilescore = profilescore.append(row_df_1,ignore_index=True)

# store onefive result
select_15_array = top_category(frame,categories,thre=1.5)
row_df_2 = get_row_result(col_name,mainframe,select_15_array)
onefive = onefive.append(row_df_2,ignore_index=True)

# store twofive result
select_25_array = top_category(frame,categories,thre=2.5)
row_df_3 = get_row_result(col_name,mainframe,select_25_array)
twofive = twofive.append(row_df_3,ignore_index=True)

# store threefive result
select_35_array = top_category(frame,categories,thre=3.5)
row_df_4 = get_row_result(col_name,mainframe,select_35_array)
threefive = threefive.append(row_df_4,ignore_index=True)

display(frame)
display(mainframe)
display(profilescore)
display(onefive)
display(twofive)
display(threefive)


In [11]:
tic = time.perf_counter()
prev_tac = tic
y=0
for i in range(1000):
    try:
        tac = time.perf_counter()
        time_compare = tac-prev_tac
        if time_compare >= 60:
            print("Connecting server")
            conn = psycopg2.connect(host="winkl-production.cpsgstz5cgos.us-west-2.rds.amazonaws.com",port="5432",database="winkl_production", user="winkl", password="Winkl911911")
            cur = conn.cursor()
            prev_tac = tac
            i = y+1
            print(prev_tac,time_compare)
    except Exception as Argument:
        print(str(Argument)+str("\n"))